In [5]:
#im2col 함수 정의
import numpy as np

def im2col(image, flt_h, flt_w, out_h, out_w):
    
    img_h, img_w = image.shape #입력 이미지의 높이와 너비
    
    #생성되는 행렬 크기
    cols = np.zeros((flt_h*flt_w, out_h*out_w))
    
    for h in range(out_h):
        h_lim = h + flt_h
        for w in range(out_w):
            w_lim = w + flt_w
            cols[:, h*out_w+w] = image[h:h_lim, w:w_lim].reshape(-1)
            
    return cols

In [6]:
#활용
img = np.array([[1, 2, 3, 4],
                [5, 6, 7, 8],
                [9, 10,11,12],
                [13,14,15,16]])
cols = im2col(img, 2, 2, 3, 3)
print(cols)

[[ 1.  2.  3.  5.  6.  7.  9. 10. 11.]
 [ 2.  3.  4.  6.  7.  8. 10. 11. 12.]
 [ 5.  6.  7.  9. 10. 11. 13. 14. 15.]
 [ 6.  7.  8. 10. 11. 12. 14. 15. 16.]]


In [7]:
#함수 수정
def im2col(image, flt_h, flt_w, out_h, out_w):
    img_h, img_w = image.shape
    cols = np.zeros((flt_h, flt_w, out_h, out_w))
    for h in range(flt_h):
        h_lim = h + out_h
        for w in range(flt_w):
            w_lim = w + out_w
            cols[h, w, :, :] = image[h:h_lim, w:w_lim]
            
    cols = cols.reshape(flt_h*flt_w, out_h*out_w)
    
    return cols

In [8]:
print(cols)

[[ 1.  2.  3.  5.  6.  7.  9. 10. 11.]
 [ 2.  3.  4.  6.  7.  8. 10. 11. 12.]
 [ 5.  6.  7.  9. 10. 11. 13. 14. 15.]
 [ 6.  7.  8. 10. 11. 12. 14. 15. 16.]]


In [11]:

def im2col(image, flt_h, flt_w, out_h, out_w):
    img_h, img_w = image.shape
    cols = np.zeros((flt_h, flt_w, out_h, out_w))
    for h in range(flt_h):
        h_lim = h + out_h
        for w in range(flt_w):
            w_lim = w + out_w
            cols[h, w, :, :] = image[h:h_lim, w:w_lim]
            
    cols= cols.reshape(flt_h*flt_w, out_h*out_w)
    
    return cols


In [12]:
print(cols)

[[ 1.  2.  3.  5.  6.  7.  9. 10. 11.]
 [ 2.  3.  4.  6.  7.  8. 10. 11. 12.]
 [ 5.  6.  7.  9. 10. 11. 13. 14. 15.]
 [ 6.  7.  8. 10. 11. 12. 14. 15. 16.]]


In [1]:
#배치와 채널을 고려한 실전 im2col 코드
def im2col(images, flt_h, flt_w, out_h, out_w):
    
    #배치 사이즈, 채널 수, 입력 이미지 높이, 너비
    n_bt, n_ch, img_h, img_w = images.shape
    
    cols = np.zeros((n_bt, n_ch, flt_h, flt_w, out_h, out_w))
    
    for h in range(flt_h):
        h_lim = h + out_h
        for w in range(flt_w):
            w_lim = w + out_w
            cols[:, :, h, w, :, :] = images[:, :, h:h_lim, w:w_lim]
            
    cols = cols.transpose(1, 2, 3, 0, 4, 5).reshape(
            n_ch*flt_h*flt_w, n_bt*out_h*out_w)
    return cols

In [7]:
#패딩과 스트라이드를 고려한 im2col 함수
def im2col(images, flt_h, flt_w, out_h, out_w, stride,pad):
    
    n_bt, n_ch, img_h, img_w = images.shape
    
    img_pad = np.pad(images, [(0,0), (0,0), (pad, pad), (pad, pad)], "constant")
    cols = np.zeros((n_bt, n_ch, flt_h, flt_w, out_h, out_w))
    
    for h in range(flt_h):
        h_lim = h + stride*out_h
        for w in range(flt_w):
            w_lim = w + stride*out_w
            cols[:, :, h, w, :, :] = img_pad[:, :, h:h_lim:stride, w:w_lim:stride]
            
    cols = cols.transpose(1, 2, 3, 0, 4, 5).reshape(n_ch*flt_h*flt_w,n_bt*out_h*out_w)
    return cols

In [9]:
import numpy as np

img = np.array([[[[1, 2, 3, 4],
                  [5, 6, 7, 8],
                  [9, 10, 11, 12],
                  [13, 14, 15, 16]]]])
cols = im2col(img, 2, 2, 3, 3, 1, 0)
print(cols)

[[ 1.  2.  3.  5.  6.  7.  9. 10. 11.]
 [ 2.  3.  4.  6.  7.  8. 10. 11. 12.]
 [ 5.  6.  7.  9. 10. 11. 13. 14. 15.]
 [ 6.  7.  8. 10. 11. 12. 14. 15. 16.]]


In [1]:
#col2im 구현 - 정의
def col2im(cols, img_shape, flt_h, flt_w, out_h, out_w, stride, pad):
    n_bt, n_ch, img_h, img_w = img_shape
    
    cols = cols.reshape(n_ch, flt_h, flt_w, n_bt, out_h,
                        out_w).transpose(3, 0, 1, 2, 4, 5)
    images = np.zeros((n_bt, n_ch, img_h+2*pad+stride-1, img_w+2*pad+stride-1))
    
    for h in range(flt_h):
        h_lim = h + stride*out_h
        for w in range(flt_w):
            w_lim = w + stride*out_w
            images[:, :, h:h_lim:stride,
                   w:w_lim:stride] += cols[:, :, h, w, :, :]
            
    return images[:, :, pad:img_h+pad, pad:img_w+pad]
                    

In [3]:
import numpy as np

cols = np.ones((4,4))
img_shape = (1, 1, 3, 3)
images = col2im(cols, img_shape, 2, 2, 2, 2, 1, 0)
print(images)

[[[[1. 2. 1.]
   [2. 4. 2.]
   [1. 2. 1.]]]]


In [4]:
#컨볼루션층 구현 - ConvLayer 클래스 생성자
class ConvLayer:
    
    #x_ch: 입력 채널 수, x_h:입력 이미지 높이, x_w:입력 이미지 너비
    #n_flt:필터 수, flt_h:필터 높이, flt_w:필터 너비
    #stride:스트라이드 너비, pad:패딩 너비
    
    def __init__(self, x_ch, x_h, x_w, n_flt, flt_h, flt_w, stride, pad):
        
        #파라미터 정리
        self.params = (x_ch, x_h, x_w, n_flt, flt_h, flt_w, stride, pad)
        
        #필터와 편향 초깃값
        self.w = wb_width * np.random.randn(n_flt, x_ch, flt_h, flt_w)
        self.b = wb_width * np.random.randn(1, n_flt)
        
        #출력 이미지 크기
        self.y_ch = n_flt #출력 채널수
        self.y_h = (x_h - flt_h + 2*pad) // stride + 1 #출력 높이
        slef.y_w = (x_w - flt_w + 2*pad) // stride + 1 #출력 너비
        